In [5]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.api.models import Collection

In [23]:
new_client = chromadb.EphemeralClient()


In [2]:
import argparse
import os
import shutil
##from langchain.document_loaders.pdf import PyPDFDirectoryLoader
##from langchain_text_splitters import RecursiveCharacterTextSplitter
##from langchain.schema.document import Document
##from langchain.vectorstores.chroma import Chroma




In [3]:
from pathlib import Path

from langchain.llms import OpenAI


from langchain_openai import OpenAIEmbeddings
##from langchain.embeddings import OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
#from langchain_community.embeddings import GPT4AllEmbeddings
from langchain import hub
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings



In [5]:
# Make sure to `pip install openai` first
from openai import OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def get_embedding(text, model="model-identifier"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

print(get_embedding("Once upon a time, there was a cat."))

[0.028939446434378624, 0.057928815484046936, -0.15416225790977478, -0.08240128308534622, 0.04368202015757561, 0.03788561746478081, -0.07805280387401581, 0.03461432084441185, -0.020030805841088295, 0.026918752118945122, -0.012021154165267944, 0.036307383328676224, 0.06827271729707718, 0.04466613754630089, -0.07091200351715088, -0.07230120152235031, 0.04448623210191727, -0.03792331740260124, 0.011466280557215214, 0.03283930942416191, 0.0076200389303267, -0.040223825722932816, 0.053429409861564636, 0.032081905752420425, 0.041595764458179474, 0.047881025820970535, -0.03246976435184479, 0.05861727148294449, -0.013796555809676647, 0.018818113952875137, 0.022399185225367546, -0.019474631175398827, -0.0037367152981460094, -0.039290908724069595, -0.012551003135740757, -0.039881631731987, 0.07455678284168243, 0.017372798174619675, 0.05405952408909798, 0.06093570217490196, 0.02510962262749672, 0.03223268687725067, -0.03783651813864708, -0.01020593661814928, 0.041464246809482574, 0.028547156602144

In [6]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
##from langchain_community.text_splitter import RecursiveCharacterTextSplitter
##from langchain_community.embeddings import OpenAIEmbeddings
##from langchain_community.vectorstores import Chroma

from pathlib import Path

# Load documents
fulldir = Path("C:/MyFolder")
dirloader = DirectoryLoader(fulldir.absolute(), glob='**/*.txt', loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
documents = dirloader.load()

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)




# Initialize embeddings
embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio"
)

# Initialize Chroma vector store
new_client = chromadb.EphemeralClient()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

print("Finished the vectorstore")


BadRequestError: Error code: 400 - {'error': "'input' field must be a string or an array of strings"}

In [7]:
# Print the types and a preview of the contents of each item in splits
for i, split in enumerate(splits):
    print(f"Element {i} is of type {type(split)}")
    if isinstance(split, str):
        print(f"Content: {split[:100]}...")  # Preview first 100 characters if it's a string
    elif hasattr(split, 'page_content'):
        print(f"Document content preview: {split.page_content[:100]}...")  # Preview first 100 characters of page_content if it's a Document
    else:
        print(f"Unknown type: {split}")


Element 0 is of type <class 'langchain_core.documents.base.Document'>
Document content preview: ﻿hello test...


In [14]:
from langchain_core.documents.base import Document  # Make sure to import the Document class

# Extract the page_content from each Document in splits
for i, split in enumerate(splits):
    if isinstance(split, Document):
        print(f"Element {i} content preview: {split.page_content[:100]}...")  # Print the first 100 characters of the content
    else:
        print(f"Element {i} is not a Document, it is of type {type(split)}")


Element 0 content preview: ﻿The Project Gutenberg eBook of Manual for the Solution of Military Ciphers
    
This ebook is for t...
Element 1 content preview: *** START OF THE PROJECT GUTENBERG EBOOK MANUAL FOR THE SOLUTION OF MILITARY CIPHERS ***




Produce...
Element 2 content preview: 1916








                        MANUAL FOR THE SOLUTION
                          OF MILITARY C...
Element 3 content preview: It is necessary therefore to fall back on ciphers for general military
work if secrecy of communicat...
Element 4 content preview: The capture of messengers is no longer the only means available to
the enemy for gaining information...
Element 5 content preview: It is not alone the body of the message which should be in cipher. It
is equally important that, dur...
Element 6 content preview: Kerckhoffs has stated that a military cipher should fulfill the
following requirements:


    1st. T...
Element 7 content preview: A brief consideration of these six conditions must lead 

In [8]:
# Convert splits to a list of strings by extracting the page_content from each Document
processed_splits = [split.page_content for split in splits if isinstance(split, Document)]

# Now processed_splits is a list of strings
print(f"Processed splits contains {len(processed_splits)} items.")
for i, content in enumerate(processed_splits[:5]):  # Print a preview of the first 5 items
    print(f"Processed split {i} content preview: {content[:100]}...")


NameError: name 'Document' is not defined

In [9]:
import numpy as np
from langchain_core.documents.base import Document  # Ensure you have the Document class imported

# Convert splits to an array of strings by extracting the page_content from each Document
array_splits = np.array([split.page_content for split in splits if isinstance(split, Document)])

# Now processed_splits is an array of strings
print(f"Processed splits contains {array_splits.size} items.")
for i, content in enumerate(array_splits[:5]):  # Print a preview of the first 5 items
    print(f"Processed split {i} content preview: {content[:100]}...")


Processed splits contains 1 items.
Processed split 0 content preview: ﻿hello test...


In [10]:

embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="nomic-ai/nomic-embed-text-v1.5-GGUF",
    # model="text-embedding-3-small",
    # embedding_ctx_length=1000,
    # tiktoken_enabled=True,
    )
new_client = chromadb.EphemeralClient()



##vectorstore = Chroma.from_documents(documents=array_splits, embedding=embeddings)

vectorstore = Chroma.from_texts(texts=splits.tolist(), embedding=embeddings)

print("finished the vectorestore")


BadRequestError: Error code: 400 - {'error': "'input' field must be a string or an array of strings"}

In [17]:
# Check if processed_splits is a list
if isinstance(processed_splits, list):
    print("processed_splits is a list.")
else:
    print("processed_splits is not a list, it is a", type(processed_splits))

# Check if all elements in processed_splits are strings
if all(isinstance(content, str) for content in processed_splits):
    print("All elements in processed_splits are strings.")
else:
    print("Not all elements in processed_splits are strings.")

# Additional debugging: Print the type of the first few elements
for i, content in enumerate(processed_splits[:5]):  # Check first 5 elements
    print(f"Element {i} is of type {type(content)}")

processed_splits is a list.
All elements in processed_splits are strings.
Element 0 is of type <class 'str'>
Element 1 is of type <class 'str'>
Element 2 is of type <class 'str'>
Element 3 is of type <class 'str'>
Element 4 is of type <class 'str'>


In [36]:


print("loading model")
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

fulldir = Path("C:/MyFolder")


# Modify TextLoader to include the encoding parameter
dirloader = DirectoryLoader(fulldir.absolute(),glob='**/*.txt',loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'}) # Specify the encoding here
##dirloader = DirectoryLoader(fulldir.absolute(), glob='**/*.txt', loader_cls=TextLoader)

##print("instantiated loader")
dirdata = dirloader.load()

# print("Data was: ", data)
print("splitting text and embedding using gpt4all embeddings")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
splits = text_splitter.split_documents(dirdata)

# Extract the text content from each Document object
splits = [doc.page_content for doc in splits]

# Ensure all splits are now strings
assert all(isinstance(text, str) for text in splits), "All splits must be strings"

embeddings = OpenAIEmbeddings(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="nomic-ai/nomic-embed-text-v1.5-GGUF",
    # model="text-embedding-3-small",
    # embedding_ctx_length=1000,
    # tiktoken_enabled=True,
    )
new_client = chromadb.EphemeralClient()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
print("finished the vectorestore")


loading model
splitting text and embedding using gpt4all embeddings


AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
# Convert splits to a list of strings by extracting the page_content from each Document
processed_splits = [split.page_content for split in splits if isinstance(split, Document)]

# Now processed_splits is a list of strings
print(f"Processed splits contains {len(processed_splits)} items.")
for i, content in enumerate(processed_splits[:5]):  # Print a preview of the first 5 items
    print(f"Processed split {i} content preview: {content[:100]}...")

In [35]:
print(splits)

['\ufeffThe Project Gutenberg eBook of Manual for the Solution of Military Ciphers\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: Manual for the Solution of Military Ciphers\n\nAuthor: Parker Hitt\n\nRelease date: May 4, 2015 [eBook #48871]\n\nLanguage: English\n\nCredits: Produced by Jeroen Hellingman and the Online Distributed\n        Proofreading Team at http://www.pgdp.net/ for Project\n        Gutenberg (This file was produced from images generously\n        made available by The Internet Archive/American Libraries.)\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK MANUAL

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = client

template = """Use the provided pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.

CONTEXT:

```{context}```

QUESTION: {question}

HELPFUL ANSWER:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def enter_question():
    print("about to invoke the rag_chain")
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )

    question = input("Enter your prompt: ")
    for chunk in rag_chain.stream(question):
        print(chunk, end="", flush=True)
    print("just finished invoking the rag_chain")
    # cleanup

while True:
    enter_question()

vectorstore.delete_collection()

In [8]:
from langchain_community.document_loaders import PyPDFLoader